# Prototype for simplifying SMS++ to PyPSA conversion

This notebook depicts the current status of the conversion between SMS++ and PyPSA using the following tools under development:
- pySMSpp: it aims to provide an abstract python interface for SMS++ models
- pypsa2smspp: it aims to provide a converter from PyPSA to SMS++ models exploiting pySMSpp

In this notebook, the following steps are performed:
1. Create a simple PyPSA model
2. Convert the PyPSA model to SMS++ using pypsa2smspp and optimize the model
3. Verify the equivalence of objective function
4. Analyze the PyPSA model created with SMS++ and pypsa2smspp and verify dispatch

## 1. Creation of the PyPSA model (from UCBlock)

We create a simple PyPSA model arbitrary number of buses, few generators, storages and loads.

Main assumptions are:
- The network is purely radial in the form bus1 -> bus2 -> bus3 -> ... busN
- A load is added to each bus of the network
- The following technologies are supported:
  - fuel-fired diesel generator
  - pv generator
  - wind generator
  - battery
  - hydro unit

In [ ]:
n_snapshots = 1*24  # number of snapshots of the model

buses_demand = [0, 1]  # list of buses where demand is located
bus_PV = 0 # Bus where PV is located; if none, no PV is considered
bus_wind = 0 # Bus where wind is located; if none, no wind is considered
bus_storage = 0 # Bus where storage is located; if none, no storage is considered
bus_hydro = None # Bus where hydro is located; if none, no hydro is considered
bus_diesel = 0 # Bus where diesel is located; if none, no pv is considered


#### Preliminary imports

In [ ]:
folder_builder = "../data/SMSpp/UCBlockSolver"
path_smspp_pypsa = folder_builder + "/pypsa2smspp.nc4"

renewable_carriers = ["pv", "wind"]

In [ ]:
import pypsa
from helpers import build_microgrid_model
import netCDF4 as nc
import pandas as pd
import numpy as np
import re
import pypsa2smspp
import pysmspp

NC_DOUBLE = "f8"
NP_DOUBLE = np.float64
NC_UINT = "u4"
NP_UINT = np.uint32

#### The following code creates the desired pypsa model.

In [ ]:
n = build_microgrid_model(
    n_snapshots = n_snapshots,
    buses_demand = buses_demand,
    bus_PV = bus_PV,
    bus_wind = bus_wind,
    bus_storage = bus_storage,
    bus_diesel = bus_diesel,
    bus_hydro=bus_hydro,
    x = 10.389754,
    y = 43.720810,
    hydro_factor=0.1,
)

# n.storage_units.capital_cost *= 2
# n.storage_units.p_min_pu = 0.
# n.storage_units.p_nom_max = 10
n.storage_units.max_hours = 1

n.snapshot_weightings["stores"] = 1.0

# n.generators.p_nom = 100

In [ ]:
n_backup = n.copy()

n.optimize(solver_name="highs")

## 2. Convert the PyPSA model to SMS++ using pypsa2smspp and solve it!

In [ ]:
tran = pypsa2smspp.Transformation()
n_smspp = tran.run(n, verbose=False)

Objective value

In [ ]:
n_smspp.objective

## 3. Verify the equivalence of SMS++ and PyPSA results

In [ ]:
pypsa_obj = n.objective
smspp_obj = n_smspp.objective
print("SMS++ obj         : %.6f" % smspp_obj)
print("PyPSA obj         : %.6f" % pypsa_obj)
print("Error SMS++ - PyPSA [%%]: %.5f" % (100*(smspp_obj - pypsa_obj)/pypsa_obj))

## 4. Analyze the PyPSA model created with SMS++ and pypsa2smspp

Retrieve PyPSA statistics

In [ ]:
n_stat = n.statistics.energy_balance(comps=["Generator"]).droplevel([0, 2])
n_stat

Retrieve SMS++ statistics

In [ ]:
n_parsed_stat = n_smspp.statistics.energy_balance(comps=["Generator"]).droplevel([0, 2])
n_parsed_stat

In [ ]:
n_stat - n_parsed_stat

In [ ]:
n_parsed_stat

In [ ]:
n_stat

In [ ]:
error_stat = n_stat - n_parsed_stat

merged_stat = pd.concat([n_stat, n_parsed_stat, error_stat], axis=1)
merged_stat.columns = ["pypsa", "smspp", "mismatch"]
merged_stat